# Подсчет числа пользовательских сессий
Вам необходимо подсчитать число пользовательских сессий в разбивке по доменам на данных из лог-файла.

**Пользовательская сессия** - это пребывание пользователя на сайте такое, что между двумя последовательными кликами проходит не более 30 минут.
Лог-файл такой же, как и на лекции. Находится в HDFS по пути `/lectures/lecture02/data/logsM.txt`

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("Spark SQL seminar").getOrCreate()

In [4]:
spark

## Задание №1
Создайте `DataFrame` из лог-файла. Схему можно скопировать из лекции.

In [5]:
from pyspark.sql.types import *

log_schema = StructType(fields=[
    StructField("ip", StringType()),
    StructField("timestamp", LongType()),
    StructField("url", StringType()),
    StructField("size", IntegerType()),
    StructField("code", IntegerType()),
    StructField("ua", StringType())
])

log = spark.read.csv("/lectures/lecture02/data/logsM.txt", sep="\t", schema=log_schema).cache()

In [6]:
log.show(5)

+--------------+--------------+--------------------+----+----+--------------------+
|            ip|     timestamp|                 url|size|code|                  ua|
+--------------+--------------+--------------------+----+----+--------------------+
| 33.49.147.163|20140101014611|http://news.rambl...| 378| 431|Safari/5.0 (compa...|
|197.72.248.141|20140101020306|http://news.mail....|1412| 203|Safari/5.0 (compa...|
| 33.49.147.163|20140101023103|http://lenta.ru/4...|1189| 451|Chrome/5.0 (compa...|
| 75.208.40.166|20140101032909|http://newsru.com...|  60| 306|Safari/5.0 (Windo...|
|197.72.248.141|20140101033626|http://newsru.com...| 736| 307|Chrome/5.0 (compa...|
+--------------+--------------+--------------------+----+----+--------------------+
only showing top 5 rows



## Задание №2
Лог не содержит столбца с доменом. Конечно можно извлечь домен с помощью функции [regexp_extract](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_extract), но мы так делать не будем. Напишите `pandas_udf`, которая будет извлекать домены из столбца `url`. Результаты применения функции поместите в столбец `domain`.

Для извлечения домена можно воспользоваться функцией [urlparse](https://docs.python.org/3/library/urllib.parse.html#urllib.parse.urlparse)

In [7]:
from urllib.parse import urlparse
import pyspark.sql.functions as f

In [8]:
def _extract_domain(url):
    return urlparse(url).netloc

In [9]:
_extract_domain(log.take(1)[0].url)

'news.rambler.ru'

In [10]:
@f.pandas_udf(StringType())
def extract_domain(urls):
    return urls.apply(_extract_domain)

In [11]:
log = log.withColumn("domain", extract_domain("url"))

In [12]:
log.show(5)

+--------------+--------------+--------------------+----+----+--------------------+---------------+
|            ip|     timestamp|                 url|size|code|                  ua|         domain|
+--------------+--------------+--------------------+----+----+--------------------+---------------+
| 33.49.147.163|20140101014611|http://news.rambl...| 378| 431|Safari/5.0 (compa...|news.rambler.ru|
|197.72.248.141|20140101020306|http://news.mail....|1412| 203|Safari/5.0 (compa...|   news.mail.ru|
| 33.49.147.163|20140101023103|http://lenta.ru/4...|1189| 451|Chrome/5.0 (compa...|       lenta.ru|
| 75.208.40.166|20140101032909|http://newsru.com...|  60| 306|Safari/5.0 (Windo...|     newsru.com|
|197.72.248.141|20140101033626|http://newsru.com...| 736| 307|Chrome/5.0 (compa...|     newsru.com|
+--------------+--------------+--------------------+----+----+--------------------+---------------+
only showing top 5 rows



## Задание №3
Для разминки давайте подсчитаем сколько дней прошло между первым и последним посещением пользователем нашего домена. Будем считать, что интересующий нас домен `news.mail.ru`. В качестве "уникального" идентификатора пользователя договоримся использовать ip-адрес. Использовать оконные функции в данном задании не надо!

### Задание №3.1
Для выполнения задания №3 понадобится делать операции с датами. Заметьте, что в столбце `timestamp` хранится не настоящий timestamp, а число с датой в формате "yyyyMMddHHmmss". Используя функции из `pyspark.sql.functions`, создайте новый столбец `timestamp`, содержащий в себе UNIX timestamp.

При возникновении ошибок, обратите внимание на типы данных. Возможно их нужно привести.

In [13]:
log = log.filter(log.domain == "news.mail.ru")

In [14]:
log = log.withColumn("timestamp", f.unix_timestamp(f.col("timestamp").cast("string"), "yyyyMMddHHmmss"))

In [15]:
log.show(5)

+--------------+----------+--------------------+----+----+--------------------+------------+
|            ip| timestamp|                 url|size|code|                  ua|      domain|
+--------------+----------+--------------------+----+----+--------------------+------------+
|197.72.248.141|1388527386|http://news.mail....|1412| 203|Safari/5.0 (compa...|news.mail.ru|
|222.131.187.37|1388533117|http://news.mail....|1017| 416|Opera/5.0 (compat...|news.mail.ru|
| 75.208.40.166|1388548882|http://news.mail....| 877| 301|Chrome/5.0 (compa...|news.mail.ru|
| 33.49.147.163|1388553876|http://news.mail....| 732| 409|Opera/5.0 (compat...|news.mail.ru|
|110.91.102.196|1388554915|http://news.mail....| 448| 201|Chrome/5.0 (compa...|news.mail.ru|
+--------------+----------+--------------------+----+----+--------------------+------------+
only showing top 5 rows



In [16]:
log.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- url: string (nullable = true)
 |-- size: integer (nullable = true)
 |-- code: integer (nullable = true)
 |-- ua: string (nullable = true)
 |-- domain: string (nullable = true)



### Задание №3.2
Приведя timestamp к правильному формату, решите исходную задачу. В результате должен получится `DataFrame` с двумя столбцами `ip` и `days`. Отсортируйте результат по столбцу `days` в порядке убывания и выведите первые 20 строк.

In [ ]:
log.groupBy("ip")\
   .agg(f.min("timestamp").alias("start"), f.max("timestamp").alias("end"))\
   .select("ip", (f.col("end") - f.col("start")).alias("diff"))\
   .select("ip", (f.col("diff") / 60 / 60 / 24).cast("int").alias("days"))\
   .orderBy("days", ascending=False)\
   .show(20)

## Задание №4
Подсчитайте число сессий, которое каждый пользователь (уникальный ip) сделал на домене `news.mail.ru`. Для решения этой задачи потребуется использование оконных функций (что это такое чуть ниже). Для работы с окнами в Spark SQL используется метод [over()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.over). Само окно определяется с помощью класса [Window](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Window). Резудьтатом будет `DataFrame` со столбцами `ip` и `sessions`, отсортированный в порядке убывания числа сессий.

_Оконная функция_ выполняет вычисления для набора строк, некоторым образом связанных с текущей строкой. Можно сравнить её с агрегатной функцией, но, в отличие от обычной агрегатной функции, при использовании оконной функции несколько строк не группируются в одну, а продолжают существовать отдельно. Внутри же, оконная функция, как и агрегатная, может обращаться не только к текущей строке результата запроса.

![](https://www.sqlitetutorial.net/wp-content/uploads/2018/11/SQLite-window-function-vs-aggregate-function.png)

Вот пример, показывающий, как сравнить зарплату каждого сотрудника со средней зарплатой его отдела:

```sql
SELECT depname, empno, salary, avg(salary) OVER (PARTITION BY depname)
  FROM empsalary;
```

```
  depname  | empno | salary |          avg          
-----------+-------+--------+-----------------------
 develop   |    11 |   5200 | 5020.0000000000000000
 develop   |     7 |   4200 | 5020.0000000000000000
 develop   |     9 |   4500 | 5020.0000000000000000
 develop   |     8 |   6000 | 5020.0000000000000000
 develop   |    10 |   5200 | 5020.0000000000000000
 personnel |     5 |   3500 | 3700.0000000000000000
 personnel |     2 |   3900 | 3700.0000000000000000
 sales     |     3 |   4800 | 4866.6666666666666667
 sales     |     1 |   5000 | 4866.6666666666666667
 sales     |     4 |   4800 | 4866.6666666666666667
(10 rows)
```

[Документация PostgreSQL](https://postgrespro.ru/docs/postgrespro/12/tutorial-window)

In [ ]:
from pyspark.sql import Window

In [ ]:
user_window = Window.orderBy("timestamp").partitionBy("ip")

In [ ]:
log.select("ip", "timestamp", f.lead("timestamp").over(user_window).alias("lead"))\
   .select("ip", "timestamp", (f.col("lead") - f.col("timestamp")).alias("diff"))\
   .where("diff >= 1800 or diff is NULL")\
   .groupBy("ip").agg(f.count("*").alias("sessions"))\
   .orderBy(f.col("sessions").desc())\
   .show(30)

## Задание №5
Нарисуйте гистограмму распределения числа сессий

In [ ]:
sessions_pd = log.select("ip", "timestamp", f.lead("timestamp").over(user_window).alias("lead"))\
                 .select("ip", "timestamp", (f.col("lead") - f.col("timestamp")).alias("diff"))\
                 .where("diff >= 1800 or diff is NULL")\
                 .groupBy("ip").agg(f.count("*").alias("sessions"))\
                 .orderBy(f.col("sessions").desc())\
                 .toPandas()

In [ ]:
sessions_pd

In [ ]:
sessions_pd.sessions.hist(bins=20)

### БОНУС. Немножечно про Arrow

In [ ]:
df = spark.range(int(1e7), numPartitions=3).cache()

In [ ]:
df.count()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "false")

In [ ]:
%%time
pdf = df.toPandas()

In [ ]:
pdf.head()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
%%time
pdf = df.toPandas()

In [ ]:
pdf.head()

In [ ]:
@f.pandas_udf(returnType=IntegerType())
def add_one(series):
    return series + 1

In [ ]:
spark.conf.get("spark.sql.execution.arrow.maxRecordsPerBatch")

In [ ]:
%%time
df.select(add_one("id")).toPandas()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 100)

In [ ]:
%%time
df.select(add_one("id")).toPandas()

In [ ]:
spark.stop()